In [1]:
#Import the libraries we need
import pandas as pd  #For reading and handling our CSV file
import numpy as np   #For working with numbers and arrays

#Let's check if our imports worked
print("Libraries loaded successfully!")

Libraries loaded successfully!


In [3]:
#Read our Sudoku puzzles from the CSV file
print("Reading the Sudoku dataset......")
sudoku_data = pd.read_csv('sudoku.csv')

#Look at what we loaded
print("\nQuick look at our dataset:")
print(f"Number of puzzles we have: {len(sudoku_data)}")
print("\nFirst few rows of our data:")
print(sudoku_data.head())

#Check if we have any missing data
print("\nDo we have any missing values?")
print(sudoku_data.isnull().sum())

Reading the Sudoku dataset......

Quick look at our dataset:
Number of puzzles we have: 1000000

First few rows of our data:
                                             quizzes  \
0  0043002090050090010700600430060020871900074000...   
1  0401000501070039605200080000000000170009068008...   
2  6001203840084590720000060050002640300700800069...   
3  4972000001004000050000160986203000403009000000...   
4  0059103080094030600275001000300002010008200070...   

                                           solutions  
0  8643712593258497619712658434361925871986574322...  
1  3461792581875239645296483719658324174729168358...  
2  6951273841384596727248369158512647392739815469...  
3  4972583161864397252537164986293815473759641828...  
4  4659123781894735623275681497386452919548216372...  

Do we have any missing values?
quizzes      0
solutions    0
dtype: int64


In [9]:
def check_puzzle_format(puzzle, solution):
    """
    Makes sure our puzzle and solution are in the correct format
    -Should be 81 characters long (9x9 grid)
    -Puzzle should only have numbers 0-9
    -Solution should only have numbers 1-9
    """
    #Check if length is correct
    if len(puzzle) != 81 or len(solution) != 81:
        print("Wrong length!")
        return False
    
    #Check if we only have valid numbers
    valid_puzzle_numbers = set('0123456789')
    valid_solution_numbers = set('123456789')
    
    puzzle_numbers = set(puzzle)
    solution_numbers = set(solution)
    
    #Return True if everything is correct
    return (puzzle_numbers.issubset(valid_puzzle_numbers) and 
            solution_numbers.issubset(valid_solution_numbers))

#Let's test our function with the first puzzle
first_puzzle = sudoku_data['quizzes'].iloc[0]
first_solution = sudoku_data['solutions'].iloc[0]

print("Testing our first puzzle:")
if check_puzzle_format(first_puzzle, first_solution):
    print("Puzzle format is correct!")
else:
    print("Something is wrong with the puzzle format")

Testing our first puzzle:
Puzzle format is correct!


In [11]:
def make_grid(puzzle_string):
    """
    Takes our puzzle string (81 characters) and makes it into a 9x9 grid
    """
    grid = []
    
    #Take 9 numbers at a time to make each row
    for i in range(0, 81, 9):
        #Get the next 9 numbers and convert them to integers
        row = [int(num) for num in puzzle_string[i:i+9]]
        grid.append(row)
    
    return grid

#Let's try it with our first puzzle
print("Our first puzzle as a grid:")
puzzle_grid = make_grid(first_puzzle)
for row in puzzle_grid:
    print(row)  #Print each row so we can see the grid structure

Our first puzzle as a grid:
[0, 0, 4, 3, 0, 0, 2, 0, 9]
[0, 0, 5, 0, 0, 9, 0, 0, 1]
[0, 7, 0, 0, 6, 0, 0, 4, 3]
[0, 0, 6, 0, 0, 2, 0, 8, 7]
[1, 9, 0, 0, 0, 7, 4, 0, 0]
[0, 5, 0, 0, 8, 3, 0, 0, 0]
[6, 0, 0, 0, 0, 0, 1, 0, 5]
[0, 0, 3, 5, 0, 8, 6, 9, 0]
[0, 4, 2, 9, 1, 0, 3, 0, 0]


In [13]:
def create_lisp_format(grid, puzzle_number):
    """
    Converts our grid into LISP format
    """
    #Start with a comment to identify the puzzle
    lisp_text = [f";; Sudoku Puzzle {puzzle_number}"]
    
    #Create the LISP variable definition
    lisp_text.append(f"(defparameter *puzzle-{puzzle_number}* '(")
    
    #Add each row of the grid
    for row in grid:
        row_text = f"  ({' '.join(map(str, row))})"
        lisp_text.append(row_text)
    
    #Close the definition
    lisp_text.append("))")
    
    #Join all lines with newlines
    return '\n'.join(lisp_text)

#Test our LISP format creator
print("Here's how our puzzle looks in LISP format:")
lisp_puzzle = create_lisp_format(puzzle_grid, 1)
print(lisp_puzzle)

Here's how our puzzle looks in LISP format:
;; Sudoku Puzzle 1
(defparameter *puzzle-1* '(
  (0 0 4 3 0 0 2 0 9)
  (0 0 5 0 0 9 0 0 1)
  (0 7 0 0 6 0 0 4 3)
  (0 0 6 0 0 2 0 8 7)
  (1 9 0 0 0 7 4 0 0)
  (0 5 0 0 8 3 0 0 0)
  (6 0 0 0 0 0 1 0 5)
  (0 0 3 5 0 8 6 9 0)
  (0 4 2 9 1 0 3 0 0)
))


In [15]:
def save_puzzles_for_lisp(puzzles, solutions, num_puzzles=100):
    """
    Creates a file with puzzles and solutions in LISP format
    We'll use a small number of puzzles to keep things simple
    """
    print(f"Creating LISP file with {num_puzzles} puzzles...")
    
    #Start our LISP file content
    lisp_content = [
        ";; Sudoku Puzzles for Our AI Project",
        ";; Each puzzle has an initial state and a goal state"
    ]
    
    #Process each puzzle
    for i in range(num_puzzles):
        #Get the puzzle and solution
        puzzle = puzzles.iloc[i]
        solution = solutions.iloc[i]
        
        #Convert to grids
        puzzle_grid = make_grid(puzzle)
        solution_grid = make_grid(solution)
        
        #Add the initial state (puzzle)
        lisp_content.append(f"\n;; Puzzle {i+1}")
        lisp_content.append(f"(defparameter *initial-state-{i+1}*")
        lisp_content.append("  '(")
        for row in puzzle_grid:
            lisp_content.append(f"    ({' '.join(map(str, row))})")
        lisp_content.append("  ))")
        
        #Add the goal state (solution)
        lisp_content.append(f"\n;; Solution {i+1}")
        lisp_content.append(f"(defparameter *goal-state-{i+1}*")
        lisp_content.append("  '(")
        for row in solution_grid:
            lisp_content.append(f"    ({' '.join(map(str, row))})")
        lisp_content.append("  ))")
    
    #Save everything to a file
    with open('sudoku_puzzles.lisp', 'w') as file:
        file.write('\n'.join(lisp_content))
    
    print("Finished! Created sudoku_puzzles.lisp")

#Create our LISP file with 100 puzzles
save_puzzles_for_lisp(
    sudoku_data['quizzes'],
    sudoku_data['solutions'],
    num_puzzles=100
)

Creating LISP file with 100 puzzles...
Finished! Created sudoku_puzzles.lisp
